In [2]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [3]:
basedir = './국토교통부_건축물대장_층별개요/'
filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&(f.startswith('floor'))]
filenames

['floor_info_baseline_seoul_201412.csv',
 'floor_info_baseline_seoul_201512.csv',
 'floor_info_baseline_seoul_201612.csv',
 'floor_info_baseline_seoul_201712.csv',
 'floor_info_baseline_seoul_201812.csv',
 'floor_info_baseline_seoul_201912.csv',
 'floor_info_baseline_seoul_202011.csv']

In [4]:
dfs_list = []
for file in filenames:
    df = pd.read_csv(basedir + file)
    dfs_list.append(df)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
first_cols = dfs_list[0].columns

In [6]:
for df in dfs_list:
    print(first_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


ValueError: Lengths must match to compare

In [7]:
dfs_list[0] = dfs_list[0].rename(columns={'건축물대장_PK':'관리_건축물대장_PK', '블럭':'블록', '동_명칭':'동_명', '면적':'면적(㎡)'})

In [8]:
dfs_list[0] = dfs_list[0].drop(columns=['마트_층별_개요_PK', '지번_좌표_X', '지번_좌표_Y', '도로명_좌표_X', '도로명_좌표_Y',
                                        '기타_기재_사항'
                                       ])

In [9]:
first_cols = dfs_list[0].columns

In [10]:
for df in dfs_list:
    print(first_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  Tr

In [11]:
concat_df = pd.concat(dfs_list)

In [12]:
concat_df.reset_index(drop=True, inplace=True)

In [13]:
concat_df.drop(columns=[col for col in concat_df.columns if col.endswith('_코드')], inplace=True)
concat_df.shape

(18712608, 24)

In [14]:
concat_df.columns = [col.replace('_코드_명', '') for col in concat_df.columns]
concat_df.head()

,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,번,지,특수지_명,블록,로트,새주소_본_번,새주소_부_번,동_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월
0,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412
1,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412
2,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412
3,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412
4,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412


In [15]:
def correct_addr(x):
    # x is a string
    if (pd.isna(x) == True)|(x == '없음'):
        return x
    else:
        x = x.replace('번지', '')
        if x[0] == ' ':
            for i in range(len(x)):
                if x[i] != ' ':
                    x = x[i:]
                    break
        
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [16]:
concat_df['지번주소'] = concat_df['대지_위치'].apply(correct_addr)

In [17]:
concat_df.drop(columns=['대지_위치', '도로명_대지_위치', '번', '지', '특수지_명', '블록', '로트', '새주소_본_번', '새주소_부_번',
                        '동_명'
                       ], inplace=True)
concat_df.head()

,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29


In [24]:
# 생각해보니 건축물에 관한 정보, 특히 사용승인일짜 같은 정보가 없으므로, 기존에 있는 living_area_per_building.csv를 쓰는 게 낫겠음.

In [18]:
concat_df['addr_split_len'] = concat_df['지번주소'].apply(lambda x: len(x.split(' ')))

In [20]:
concat_df['addr_split_len'].value_counts()

4    18624900
5       65236
6       17162
7        4197
8         730
3         291
9          92
Name: addr_split_len, dtype: int64

In [21]:
concat_df = concat_df[(concat_df['addr_split_len'] == 4)|(concat_df['addr_split_len'] == 5)]
concat_df['addr_split_len'].value_counts()

4    18624900
5       65236
Name: addr_split_len, dtype: int64

In [22]:
len5_df = concat_df[concat_df['addr_split_len'] == 5]

In [23]:
len5_df['fourth_addr'] = len5_df['지번주소'].apply(lambda x: x.split(' ')[3])

In [24]:
def landtype_name(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if x == '산':
            return '산'
        elif ('블록' in x)&('지구' in x):
            return '지구블록'
        elif '블록' in x:
            return 'BL'
        else:
            return x

In [25]:
len5_df['대장구분명'] = len5_df['fourth_addr'].apply(landtype_name)

In [27]:
len5_df['대장구분명'].value_counts()

산       51148
BL      13877
지구블록      211
Name: 대장구분명, dtype: int64

In [28]:
not_san_df = len5_df[len5_df['대장구분명'] != '산']
not_san_df.shape

(14088, 18)

In [29]:
concat_df.drop(not_san_df.index, inplace=True)

In [30]:
san_df = len5_df[len5_df['대장구분명'] == '산']
san_df.shape

(51148, 18)

In [31]:
def san_new_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split(' ')
        return splitted[0] + ' ' + splitted[1] + ' ' + splitted[2] + ' ' + splitted[3] + splitted[4]

In [32]:
san_df['지번주소'] = san_df['지번주소'].apply(san_new_addr)

In [33]:
concat_df.loc[san_df.index, '지번주소'] = san_df['지번주소']

In [34]:
def weird_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        try:
            int(x.split(' ')[-1].replace('-', ''))
            return 0
        except:
            return 1

In [35]:
len4_df = concat_df[concat_df['addr_split_len'] == 4]
len4_df['weird_addr'] = len4_df['지번주소'].apply(weird_addr)

In [36]:
weird_addr_df = len4_df[len4_df['weird_addr'] == 1]

In [37]:
concat_df.drop(weird_addr_df.index, inplace=True)

In [38]:
concat_df.shape

(18672306, 16)

In [39]:
main_data_df = pd.read_csv('./prepped_data/main_data_ver_2.5.csv')
print(main_data_df.shape)
main_data_df.head()

(4559866, 16)


,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
2,매매,67.28,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
3,매매,67.28,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
4,매매,67.28,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0


In [40]:
inter_df = concat_df[concat_df['지번주소'].isin(main_data_df['지번주소'].unique())]
print(inter_df.shape)
inter_df.head()

(6977287, 16)


,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소,addr_split_len
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4


In [41]:
%%time
inter_df.to_csv('floor_info_maindata_inter_baseline.csv', index=False)

Wall time: 46.6 s


In [42]:
inter_df.reset_index(drop=True, inplace=True)

In [43]:
structure_df = inter_df[['지번주소', '기준년월', '구조']]

In [46]:
prac_df = structure_df.iloc[:500,:]

In [49]:
for structure in prac_df['구조'].unique():
    prac_df['구조_'+structure] = prac_df['구조'].apply(lambda x: 1 if x == structure else 0)

In [50]:
prac_df.head()

,지번주소,기준년월,구조,구조_철근콘크리트구조,구조_벽돌구조,구조_철골콘크리트구조
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0


In [51]:
for structure in prac_df['구조'].unique():
    prac_df['구조_'+structure+'_sum'] = prac_df[['지번주소', '기준년월', '구조_'+structure]].groupby(['지번주소', '기준년월']).transform('sum')['구조_'+structure]

In [52]:
prac_df.head()

,지번주소,기준년월,구조,구조_철근콘크리트구조,구조_벽돌구조,구조_철골콘크리트구조,구조_철근콘크리트구조_sum,구조_벽돌구조_sum,구조_철골콘크리트구조_sum
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0


In [55]:
sum_list = [col for col in prac_df.columns if col.endswith('_sum')]
sum_list

['구조_철근콘크리트구조_sum', '구조_벽돌구조_sum', '구조_철골콘크리트구조_sum']

In [139]:
def mode_structure(x):
    # x is a groupby dataframe
    
    return x[sum_list].idxmax(axis=1)

In [140]:
prac_df['최빈구조'] = prac_df[['지번주소', '기준년월'] + sum_list].groupby(['지번주소', '기준년월']).apply(mode_structure).reset_index(drop=True)
prac_df['최빈구조'] = prac_df['최빈구조'].apply(lambda x: x.replace('_sum', '').replace('구조_', ''))

In [141]:
prac_df.head()

,지번주소,기준년월,구조,구조_철근콘크리트구조,구조_벽돌구조,구조_철골콘크리트구조,구조_철근콘크리트구조_sum,구조_벽돌구조_sum,구조_철골콘크리트구조_sum,최빈구조
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0,철근콘크리트구조
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0,철근콘크리트구조
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0,철근콘크리트구조
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0,철근콘크리트구조
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,5,0,0,철근콘크리트구조


In [142]:
prac_df['구조'].value_counts()

철근콘크리트구조    483
벽돌구조         15
철골콘크리트구조      2
Name: 구조, dtype: int64

In [143]:
prac_df['최빈구조'].value_counts()

철근콘크리트구조    492
벽돌구조          8
Name: 최빈구조, dtype: int64

In [149]:
structure_df['구조'].fillna(structure_df['구조'].mode()[0], inplace=True)

In [151]:
for structure in structure_df['구조'].unique():
    structure_df['구조_'+structure] = structure_df['구조'].apply(lambda x: 1 if x == structure else 0)

In [152]:
for structure in tqdm(structure_df['구조'].unique()):
    structure_df['구조_'+structure+'_sum'] = structure_df[['지번주소', '기준년월', '구조_'+structure]].groupby(['지번주소', '기준년월']).transform('sum')['구조_'+structure]

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:43<00:00,  2.18s/it]


In [156]:
sum_list = [col for col in structure_df.columns if col.endswith('_sum')]

In [158]:
%%time
structure_df['최빈구조'] = structure_df[['지번주소', '기준년월'] + sum_list].groupby(['지번주소', '기준년월']).apply(mode_structure).reset_index(drop=True)

Wall time: 6min 13s


In [159]:
structure_df.head()

,지번주소,기준년월,구조,구조_철근콘크리트구조,구조_벽돌구조,구조_철골콘크리트구조,구조_기타조적구조,구조_경량철골구조,구조_철골철근콘크리트구조,구조_일반철골구조,구조_기타강구조,구조_프리케스트콘크리트구조,구조_블록구조,구조_기타구조,구조_기타콘크리트구조,구조_강파이프구조,구조_일반목구조,구조_조적구조,구조_기타철골철근콘크리트구조,구조_석구조,구조_철골철근콘크리트합성구조,구조_라멘조,구조_조립식판넬조,구조_철근콘크리트구조_sum,구조_벽돌구조_sum,구조_철골콘크리트구조_sum,구조_기타조적구조_sum,구조_경량철골구조_sum,구조_철골철근콘크리트구조_sum,구조_일반철골구조_sum,구조_기타강구조_sum,구조_프리케스트콘크리트구조_sum,구조_블록구조_sum,구조_기타구조_sum,구조_기타콘크리트구조_sum,구조_강파이프구조_sum,구조_일반목구조_sum,구조_조적구조_sum,구조_기타철골철근콘크리트구조_sum,구조_석구조_sum,구조_철골철근콘크리트합성구조_sum,구조_라멘조_sum,구조_조립식판넬조_sum,최빈구조
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum


In [160]:
%%time
structure_df['최빈구조2'] = structure_df[['지번주소', '기준년월', '구조']].groupby(['지번주소', '기준년월']).transform(lambda x: x.mode()[0])['구조']

Wall time: 8min 7s


In [164]:
structure_df.head()

,지번주소,기준년월,구조,구조_철근콘크리트구조,구조_벽돌구조,구조_철골콘크리트구조,구조_기타조적구조,구조_경량철골구조,구조_철골철근콘크리트구조,구조_일반철골구조,구조_기타강구조,구조_프리케스트콘크리트구조,구조_블록구조,구조_기타구조,구조_기타콘크리트구조,구조_강파이프구조,구조_일반목구조,구조_조적구조,구조_기타철골철근콘크리트구조,구조_석구조,구조_철골철근콘크리트합성구조,구조_라멘조,구조_조립식판넬조,구조_철근콘크리트구조_sum,구조_벽돌구조_sum,구조_철골콘크리트구조_sum,구조_기타조적구조_sum,구조_경량철골구조_sum,구조_철골철근콘크리트구조_sum,구조_일반철골구조_sum,구조_기타강구조_sum,구조_프리케스트콘크리트구조_sum,구조_블록구조_sum,구조_기타구조_sum,구조_기타콘크리트구조_sum,구조_강파이프구조_sum,구조_일반목구조_sum,구조_조적구조_sum,구조_기타철골철근콘크리트구조_sum,구조_석구조_sum,구조_철골철근콘크리트합성구조_sum,구조_라멘조_sum,구조_조립식판넬조_sum,최빈구조,최빈구조2,구조_mapped
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0


In [161]:
structure_map_df = pd.DataFrame({'cat':structure_df['구조'].unique(), 'map':[i for i in range(len(structure_df['구조'].unique()))]})
structure_map_df.to_csv('./feature_maps/structure_map.csv', index=False)
structure_map = {}
for i in range(structure_map_df.shape[0]):
    structure_map[structure_map_df['cat'][i]] = structure_map_df['map'][i]

In [162]:
inverse_structure_map = {v:k for k, v in structure_map.items()}

In [163]:
structure_df['구조_mapped'] = structure_df['구조'].map(structure_map)

In [166]:
%%time
structure_df['최빈구조3'] = structure_df[['지번주소', '기준년월', '구조_mapped']].groupby(['지번주소', '기준년월']).transform(lambda x: x.mode()[0])['구조_mapped']
structure_df['최빈구조3'] = structure_df['최빈구조3'].map(inverse_structure_map)

Wall time: 7min 42s


In [168]:
structure_df.head()

,지번주소,기준년월,구조,구조_철근콘크리트구조,구조_벽돌구조,구조_철골콘크리트구조,구조_기타조적구조,구조_경량철골구조,구조_철골철근콘크리트구조,구조_일반철골구조,구조_기타강구조,구조_프리케스트콘크리트구조,구조_블록구조,구조_기타구조,구조_기타콘크리트구조,구조_강파이프구조,구조_일반목구조,구조_조적구조,구조_기타철골철근콘크리트구조,구조_석구조,구조_철골철근콘크리트합성구조,구조_라멘조,구조_조립식판넬조,구조_철근콘크리트구조_sum,구조_벽돌구조_sum,구조_철골콘크리트구조_sum,구조_기타조적구조_sum,구조_경량철골구조_sum,구조_철골철근콘크리트구조_sum,구조_일반철골구조_sum,구조_기타강구조_sum,구조_프리케스트콘크리트구조_sum,구조_블록구조_sum,구조_기타구조_sum,구조_기타콘크리트구조_sum,구조_강파이프구조_sum,구조_일반목구조_sum,구조_조적구조_sum,구조_기타철골철근콘크리트구조_sum,구조_석구조_sum,구조_철골철근콘크리트합성구조_sum,구조_라멘조_sum,구조_조립식판넬조_sum,최빈구조,최빈구조2,구조_mapped,최빈구조3
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0,철근콘크리트구조
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0,철근콘크리트구조
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0,철근콘크리트구조
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0,철근콘크리트구조
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,구조_철근콘크리트구조_sum,철근콘크리트구조,0,철근콘크리트구조


In [169]:
structure_df = structure_df[['지번주소', '기준년월', '최빈구조3']]
structure_df.head()

,지번주소,기준년월,최빈구조3
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조


In [170]:
print(structure_df.shape)
structure_df.drop_duplicates(inplace=True)
structure_df.shape

(6977287, 3)


(719064, 3)